Импортируем библиотеки, даем доступ к нашему Google Disk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from imutils import paths
from PIL import Image
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, SubsetRandomSampler, Subset,Dataset
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Загружаем предварительно обученную модель ResNet-34 из PyTorch Hub - централизованного хранилища предварительно обученных моделей, предоставляемого PyTorch.

1. torch.hub.load(): Эта функция позволяет загружать предварительно обученные модели из PyTorch Hub. Она принимает на вход три аргумента:
   - 'pytorch/vision': Это указывает на модуль, содержащий предварительно обученные модели, в данном случае - модели компьютерного зрения.
   - 'resnet34': Это название модели, которую нужно загрузить, в данном случае - ResNet-34.
   - pretrained=True: Этот аргумент указывает, что нужно загрузить предварительно обученную версию модели, а не инициализировать ее случайными весами.

In [ ]:
model_resnet34 = torch.hub.load('pytorch/vision', 'resnet34', pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_main
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Замораживаем (отключения обновления) все параметры, кроме параметров слоев BatchNorm, в предварительно обученной модели ResNet-34.

1. for name, param in model_resnet34.named_parameters(): - Этот цикл проходит по всем параметрам (весам и смещениям) в модели model_resnet34 и получает для каждого параметра его имя (name) и само значение параметра (param).

2. if("bn" not in name): - Этот условный блок проверяет, содержится ли в имени параметра (name) подстрока "bn". Если нет, то это означает, что данный параметр не относится к слоям BatchNorm.

3. param.requires_grad = False - Эта строка устанавливает флаг requires_grad для текущего параметра в False. Это означает, что данный параметр не будет обновляться во время обучения модели, так как его градиент не будет вычисляться.

Это делается для того, чтобы использовать предварительно обученную модель в качестве "базы" и обучать только нерегуляризированные слои, такие как BatchNorm. Это может помочь улучшить производительность модели, особенно когда размер обучающей выборки относительно мал.

In [ ]:
for name, param in model_resnet34.named_parameters():
    if("bn" not in name):
        param.requires_grad = False

1. Устанавливаем количество классов. Тоесть модель будет использоваться для классификации на 6 классах.

2. Модификация последнего полносвязного слоя (fully connected layer) модели ResNet-34:
   - model_resnet34.fc = nn.Sequential(...) - заменяет последний полносвязный слой в оригинальной модели ResNet-34 на новую последовательность операций.
   - nn.Linear(model_resnet34.fc.in_features, 512) - создает новый полносвязный слой, который принимает количество входных признаков, равное размеру выходного вектора предыдущего слоя в модели ResNet-34, и выдает 512 выходов.
   - nn.ReLU() - добавляет слой активации ReLU. ReLU возвращает ноль, если входное значение x меньше или равно нулю, и возвращает само входное значение x, если оно больше нуля.
   - nn.Dropout() - добавляет слой dropout для регуляризации.
   - nn.Linear(512, num_classes) - создает последний полносвязный слой, который преобразует 512 входных признаков в num_classes выходов (в данном случае 6).

Таким образом, этот код модифицирует последний слой предварительно обученной модели ResNet-34, чтобы она могла выполнять классификацию на 6 классов вместо оригинальных 1000 классов, на которых она была обучена.

In [ ]:
num_classes = 6

model_resnet34.fc = nn.Sequential(nn.Linear(model_resnet34.fc.in_features,512),
                                  nn.ReLU(),
                                  nn.Dropout(),
                                  nn.Linear(512, num_classes))

Определяем функцию train(), которая выполняет обучение модели.

1. Инициализация переменных:
   - training_loss и valid_loss - переменные для хранения суммарных значений потерь во время обучения и валидации соответственно.
   - model.train() переводит модель в режим обучения, чтобы включить такие операции, как dropout, batch normalization и т.д.

2. Обучение модели:
   - Проходит по батчам данных из train_loader.
   - Для каждого батча:
     - Обнуляет градиенты в оптимизаторе с помощью optimizer.zero_grad().
     - Получает входные данные inputs и целевые значения targets, перемещая их на выбранное устройство (device).
     - Вычисляет выходы модели output = model(inputs).
     - Вычисляет функцию потерь loss = loss_fn(output, targets).
     - Вычисляет градиенты loss.backward().
     - Обновляет параметры модели с помощью optimizer.step().
     - Суммирует значение потерь training_loss += loss.data.item() * inputs.size(0).
   - Вычисляет усредненное значение потерь за эпоху training_loss /= len(train_loader.dataset).

3. Переключение модели в режим оценки: model.eval() переводит модель в режим оценки, отключая такие операции, как dropout и batch normalization, которые нужны только во время обучения.

4. Инициализация счетчиков:
   - num_correct - количество правильно предсказанных примеров.
   - num_examples - общее количество примеров в валидационном наборе.

5. Проход по батчам валидационного набора:
   - Для каждого батча:
     - Получает входные данные inputs и целевые значения targets, перемещая их на выбранное устройство (device).
     - Вычисляет выходы модели output = model(inputs).
     - Вычисляет значение функции потерь loss = loss_fn(output, targets).
     - Суммирует значение потерь valid_loss += loss.data.item() * inputs.size(0).
     - Вычисляет количество правильно предсказанных примеров в батче:
       - Находит индексы максимальных значений в выходе модели torch.max(F.softmax(output, dim=1), dim=1)[1].
       - Сравнивает эти индексы с целевыми значениями targets и считает количество совпадений torch.eq().
       - Суммирует количество правильных предсказаний num_correct += torch.sum(correct).item().
       - Увеличивает количество примеров num_examples += correct.shape[0].

6. Вычисление усредненного значения потерь:
   - Делит суммарное значение потерь на количество примеров в валидационном наборе valid_loss /= len(val_loader.dataset).

В итоге, этот код вычисляет две важные метрики для оценки модели на валидационном наборе:
1. Значение функции потерь valid_loss.
2. Точность модели num_correct / num_examples.

In [ ]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=15, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)


        model.eval()
        num_correct = 0
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets)
            valid_loss += loss.data.item() * inputs.size(0)

            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

Делаем преобразования данных.

1. Преобразования для обучения ('train'):
   - transforms.RandomResizedCrop(224): Случайно изменяет размер и обрезает изображение до размера 224x224 пикселей.
   - transforms.RandomHorizontalFlip(): Случайно отражает изображение по горизонтали.
   - transforms.RandomRotation(degrees=15): Случайно поворачивает изображение на угол до 15 градусов.
   - transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2): Случайно изменяет яркость, контраст и насыщенность цвета изображения.
   - transforms.ToTensor(): Преобразует изображение в тензор PyTorch.
   - transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]): Нормализует изображение с использованием средних и стандартных отклонений, характерных для набора данных ImageNet.

2. Преобразования для тестирования ('test'):
   - transforms.Resize(256): Изменяет размер изображения до 256x256 пикселей.
   - transforms.CenterCrop(224): Вырезает центральный фрагмент 224x224 пикселей.
   - transforms.ToTensor(): Преобразует изображение в тензор PyTorch.
   - transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]): Нормализует изображение с использованием тех же средних и стандартных отклонений, что и для обучения.

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

1. Задание пути к данным:
   - data_dir = '/content/drive/My Drive/contest/data_contest/train': Устанавливает путь к директории, содержащей тренировочные данные.

2. Создание датасетов:
   - image_datasets = {'train': datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train'])}: Создает датасет ImageFolder для тренировочных данных, применяя к ним преобразования из data_transforms['train'].

3. Получение списка изображений и их меток:
   - image_folder = image_datasets['train'].imgs: Получает список всех изображений в тренировочном наборе.
   - labels = [x[1] for x in image_folder]: Получает список меток (классов) для всех изображений.

4. Разделение данных на тренировочную и валидационную выборки:
   - train_indices, val_indices = train_test_split(range(len(image_folder)), test_size=0.2, random_state=42): Разделяет индексы изображений на тренировочную (80%) и валидационную (20%) выборки.

5. Создание выборок для тренировки и валидации:
   - train_sampler = SubsetRandomSampler(train_indices): Создает выборку для тренировки, используя случайные индексы из train_indices.
   - val_sampler = SubsetRandomSampler(val_indices): Создает выборку для валидации, используя случайные индексы из val_indices.

6. Создание загрузчиков данных (dataloaders):
   - train_loader = DataLoader(image_datasets['train'], batch_size=16, sampler=train_sampler, num_workers=6): Создает загрузчик для тренировочных данных, используя train_sampler для формирования батчей.
   - val_loader = DataLoader(image_datasets['train'], batch_size=16, sampler=val_sampler, num_workers=6): Создает загрузчик для валидационных данных, используя val_sampler для формирования батчей.
   - dataloaders = {'train': DataLoader(image_datasets['train'], batch_size=16, shuffle=True, num_workers=6)}: Создает загрузчик для тренировочных данных с перемешиванием.

7. Сохранение информации о датасете:
   - dataset_sizes = {'train': len(image_datasets['train'])}: Сохраняет размер тренировочного набора.
   - class_names = image_datasets['train'].classes: Сохраняет названия классов.

Этот код готовит данные для обучения и валидации модели глубокого обучения, используя PyTorch. Он разделяет исходный набор данных на тренировочную и валидационную выборки, создает загрузчики данных, которые будут использоваться в процессе обучения

In [ ]:
data_dir = '/content/drive/My Drive/contest/data_contest/train'
image_datasets = {'train': datasets.ImageFolder(os.path.join(data_dir, 'train'),
                                          data_transforms['train'])}
image_folder = image_datasets['train'].imgs
labels = [x[1] for x in image_folder]
train_indices, val_indices = train_test_split(range(len(image_folder)), test_size=0.2, random_state=42)

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(image_datasets['train'], batch_size=16, sampler=train_sampler, num_workers=6)
val_loader = DataLoader(image_datasets['train'], batch_size=16, sampler=val_sampler, num_workers=6)

dataloaders = {'train': DataLoader(image_datasets['train'], batch_size=16,
                             shuffle=True, num_workers=6)}
dataset_sizes = {'train': len(image_datasets['train'])}
class_names = image_datasets['train'].classes

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Определяем настраиваемый датасет для тестовых данных и создаем загрузчик данных (DataLoader) для него.

1. Определение класса CustomTestDataset:
   - def __init__(self, image_paths, transform=None):: Конструктор класса, который принимает список путей к изображениям и необязательное преобразование для применения к изображениям.
   - def __len__(self):: Возвращает количество изображений в датасете.
   - def __getitem__(self, idx):: Возвращает изображение по указанному индексу. Он открывает файл изображения, преобразует его в формат RGB и применяет к нему заданное преобразование (если оно указано).

2. Получение путей к тестовым изображениям:
   - image_paths = sorted(list(paths.list_images('/content/drive/My Drive/contest/data_contest/test/test'))): Получает отсортированный список полных путей ко всем изображениям в директории /content/drive/My Drive/contest/data_contest/test/test.

3. Создание экземпляра CustomTestDataset:
   - custom_test_dataset = CustomTestDataset(image_paths, transform=data_transforms['test']): Создает экземпляр CustomTestDataset, используя список путей к изображениям и преобразования из data_transforms['test'].

4. Создание DataLoader для тестовых данных:
   - test_loader = DataLoader(custom_test_dataset, batch_size=16, shuffle=False, num_workers=6): Создает загрузчик данных для тестового датасета, используя размер батча 16, без перемешивания и с 6 рабочими процессами.

5. Обновление словаря dataloaders и dataset_sizes:
   - dataloaders['test'] = test_loader: Добавляет созданный загрузчик тестовых данных в словарь dataloaders.
   - dataset_sizes['test'] = len(image_paths): Добавляет размер тестового датасета в словарь dataset_sizes.

Этот код позволяет загружать и обрабатывать тестовые данные с помощью пользовательского датасета CustomTestDataset, который применяет к изображениям такие же преобразования, как и для тренировочных данных. Созданный загрузчик данных test_loader может быть использован для оценки модели на тестовом наборе.

In [ ]:
class CustomTestDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path.strip())
        image = image.convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

image_paths = sorted(list(paths.list_images('/content/drive/My Drive/contest/data_contest/test/test')))

custom_test_dataset = CustomTestDataset(image_paths, transform=data_transforms['test'])

test_loader = DataLoader(custom_test_dataset, batch_size=16, shuffle=False, num_workers=6)
dataloaders['test'] = test_loader
dataset_sizes['test'] = len(image_paths)

1. Проверка доступности CUDA:
   - if torch.cuda.is_available():: Проверяет, доступен ли CUDA на текущем устройстве.

2. Установка устройства:
   - device = torch.device("cuda"): Если CUDA доступен, устанавливает устройство в качестве CUDA.
   - device = torch.device("cpu"): Если CUDA не доступен, устанавливает устройство в качестве CPU.

   GPU (графические процессоры) обычно намного производительнее CPU (центральных процессоров) при выполнении параллельных вычислений.

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

Информация о данных

In [ ]:
val_loader.dataset

Dataset ImageFolder
    Number of datapoints: 180
    Root location: /content/drive/My Drive/contest/data_contest/train/train
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=True)
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-15.0, 15.0], interpolation=nearest, expand=False, fill=0)
               ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=(0.8, 1.2), hue=None)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

Оценка производительности обученной модели на тестовом наборе данных.

1. Инициализация счетчиков:
   - correct = 0: Инициализирует счетчик правильных предсказаний.
   - total = 0: Инициализирует счетчик общего количества тестовых примеров.

2. Оценка модели в режиме без градиента:
   - with torch.no_grad():: Этот блок выполняется без вычисления градиентов, что повышает эффективность, поскольку градиенты не нужны во время тестирования.

3. Итерация по тестовому загрузчику данных:
   - for data in test_loader:: Итерируется по батчам тестовых данных, загруженных через test_loader.
   - images, labels = data[0].to(device), data[1].to(device): Извлекает изображения и соответствующие им метки классов из текущего батча и переносит их на выбранное устройство (CPU или GPU).

4. Оценка модели и подсчет правильных предсказаний:
   - outputs = model(images): Получает выходы модели для текущего батча изображений.
   - _, predicted = torch.max(outputs.data, 1): Получает индексы классов с максимальными выходными значениями, что соответствует предсказанным классам.
   - total += labels.size(0): Увеличивает счетчик общего количества тестовых примеров на размер текущего батча.
   - correct += (predicted == labels).sum().item(): Увеличивает счетчик правильных предсказаний на количество примеров в текущем батче, для которых предсказанный класс совпадает с истинным классом.

5. Вывод результатов:
   - print('correct: {:d}  total: {:d}'.format(correct, total)): Выводит количество правильных предсказаний и общее количество тестовых примеров.
   - print('accuracy = {:f}'.format(correct / total)): Вычисляет и выводит точность модели на тестовом наборе данных.

Функция test_model оценивает производительность обученной модели на тестовом наборе данных и выводит общее количество правильных предсказаний, а также точность модели.


In [ ]:
model_resnet34.eval()
test_predictions = []
def test_model(model):
    with torch.no_grad():
        for data in test_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            test_predictions.extend(predicted.cpu().numpy())

1. Переносит модель на устройство:
   - model_resnet34.to(device): Переносит модель ResNet-34 на устройство, указанное в переменной device. Это может быть либо CPU, либо GPU (если доступен и выбран).
   - Это необходимо, чтобы все вычисления, связанные с моделью, выполнялись на том же устройстве, что и тренировочные и валидационные данные.

2. Создает оптимизатор:
   - optimizer = optim.Adam(model_resnet34.parameters(), lr=0.001): Создает оптимизатор Adam с начальной скоростью обучения 0.001. Оптимизатор будет обновлять параметры модели во время обучения.

3. Вызывает функцию train:
   - train(model_resnet34, optimizer, torch.nn.CrossEntropyLoss(), train_loader, val_loader, epochs=15, device=device): Вызывает функцию train, которая, вероятно, определена в другом месте кода. Эта функция отвечает за обучение модели ResNet-34.
   - Она принимает следующие аргументы:
     - model_resnet34: Экземпляр модели ResNet-34.
     - optimizer: Оптимизатор, который будет использоваться для обновления параметров модели.
     - torch.nn.CrossEntropyLoss(): Функция потерь, которая будет использоваться для оценки производительности модели.
     - train_loader: Загрузчик данных для обучающего набора данных.
     - val_loader: Загрузчик данных для валидационного набора данных.
     - epochs=15: Число эпох, на протяжении которых будет производиться обучение модели.
     - device=device: Устройство, на котором будут выполняться вычисления (CPU или GPU).

Таким образом, этот код готовит модель ResNet-34 к обучению, создает оптимизатор и вызывает функцию обучения, используя заданные параметры и загрузчики данных. Обучение будет выполняться на том же устройстве, что и тренировочные данные.


In [ ]:
model_resnet34.to(device)
optimizer = optim.Adam(model_resnet34.parameters(), lr=0.001)
train(model_resnet34, optimizer, torch.nn.CrossEntropyLoss(), train_loader, val_loader, epochs=15, device=device)

1. Создает словарь heroes, который сопоставляет числовые идентификаторы с соответствующими именами героев.

2. Создает pandas DataFrame res_df, используя результаты предсказаний test_predictions. Столбец my_class содержит числовые идентификаторы предсказанных классов.

3. Применяет функцию lambda к столбцу my_class DataFrame res_df. Эта функция использует словарь heroes для замены числовых идентификаторов на соответствующие имена героев. Результат записывается в новый столбец my_class.

4. Извлекает пути к изображениям из test_loader.dataset.image_paths и создает новый столбец path в DataFrame res_df, содержащий только имена файлов (без полного пути).

5. Добавляет новый столбец id с последовательными числовыми идентификаторами от 0 до 599 (предполагается, что в тестовом наборе 600 изображений).

6. Переименовывает столбец my_class в class.

7. Сохраняет DataFrame res_df в файл pred2.csv в формате CSV без индексов.

В целом, этот код создает табличное представление результатов предсказаний модели, связывая предсказанные классы с именами героев, добавляя пути к изображениям и последовательные идентификаторы, а затем сохраняет это представление в CSV-файл.

In [ ]:
heroes = {0: 'Гароу', 1: 'Генос', 2: 'Сайтама', 3: 'Соник', 4: 'Татсумаки', 5: 'Фубуки'}
res_df = pd.DataFrame(test_predictions, columns=['my_class'])
res_df['my_class'] = res_df.apply(lambda item: heroes[item.my_class], axis=1)
image_path = [path.split(os.path.sep)[-1] for path in test_loader.dataset.image_paths]
res_df.insert(0, "path", image_path, True)
res_df.insert(0, 'id', range(600), True)
res_df.rename(columns={'my_class': 'class'}, inplace=True)
res_df.to_csv('pred2.csv', index=False)
